In [ ]:
import re

import pandas as pd
from pathlib import Path
from ploting import *

In [ ]:
root_path = "crusher"

In [ ]:
name_re = re.compile("(weak|strong)_scaling-(\d+pt)-n_(\d+)-([\w_]+)_comm-(gpu|cpu)-nodes_(\d+)")

In [ ]:
time_re = re.compile("DURATION:\s*(\d+(\.\d+)?(e[+-]\d+)?)")

In [ ]:
size_re = re.compile("SIZE:\s*(\d+)")

In [ ]:
stencil_type = dict([((True, 2), "5pt"), ((True, 3), "7pt"), ((False, 2), "9pt"), ((False, 3), "27pt")])

In [ ]:
num_gpus = 8

In [ ]:
data = []
for file in Path(f"{root_path}/data").iterdir():
    if m := name_re.search(file.name):
        with open(file, "r") as input:
            input_str = input.read()
            duration = 0
            size = 0
            if t_m := time_re.search(input_str):
                duration = t_m.group(1)
            if s_m := size_re.search(input_str):
                size = s_m.group(1)
            data.append({"scaling": m.group(1), "stencil": m.group(2), "n": m.group(3),
                         "comm_pattern": m.group(4),
                         "type": m.group(5), "nodes": m.group(6), "time": duration, "size": size})
df = pd.DataFrame(data).drop(columns=["scaling", "type"]).astype({"n": int, "nodes": int, "time": float, "size": int})
df

In [ ]:
df["gpus"] = df["nodes"] * num_gpus

In [ ]:
piv = df.drop(columns=["size", "nodes"]).pivot(index="gpus", columns=["stencil", "comm_pattern", "n"], values="time")
piv = piv[piv.columns.sort_values()]
piv

In [ ]:
piv["5pt"]

In [ ]:
df["flop"] = df["size"] * (df.stencil.apply(lambda s: int(s[:-2])) *2 + 12)
df["flop/s"] = df.flop / df.time
df

In [ ]:
piv_flops = df[df.n >= 100000].drop(columns=["size", "nodes", "flop", "time"]).pivot(index="gpus", columns=["comm_pattern", "n", "stencil"], values="flop/s")
piv_flops = piv_flops[piv_flops.columns.sort_values()]
piv_flops["optimal"][1000000]

In [ ]:
_df = piv_flops["optimal"][1000000]
linear_scaling_x = [1] + list(_df.index) + [2000]
linear_scaling = [_df["27pt"][8] * 1.2 / 1e12 * i / _df.index[0] for i in linear_scaling_x]
linear_scaling

In [ ]:
4.624524e+13 * 3

In [ ]:
linear_scaling_x

In [ ]:
_df = piv_flops["optimal"][1000000]
fig, ax = plt.subplots(dpi=300)
ax.set_prop_cycle(default_cycler)
ax.set_title(f"CG Performance per Iteration")
(_df / 1e12).plot(ax=ax, legend=True, logx=True, logy=True)
xlim = ax.get_xlim()
ylim = ax.get_ylim()
ax.plot(linear_scaling_x, linear_scaling, '--', color="grey", )
ax.legend([n for n in piv_flops.columns.levels[2]] + ["Ideal"])
ax.set_xlim([xlim[0] * 0.85, xlim[1] * 1.15])
ax.set_ylim([ylim[0] * 0.85, ylim[1] * 1.15])
ax.set_xlabel("# GPUs")
ax.set_ylabel("TFLOP/s")
fig.savefig(f"{root_path}/img/optimal-flops.png")

In [ ]:
"3pt"[:-2]